In [2]:
import os

import pandas as pd

In [21]:
import datetime

import fasttext
import keras_core as keras
import numpy as np
import pandas as pd
from keras_core import Sequential
from keras_core.src.layers import Dense
from keras_core.models import load_model
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import tensorflow as tf

from tensorboard.plugins.hparams import api as hp

## import fastTest model

In [13]:
import os
#embedded_path = './../../'

#contents = os.listdir(embedded_path)
#print(contents)
ft = fasttext.load_model('../../data/embedding_data/cc.fr.300.bin')

['notebooks', 'tmp', 'tests', '.env', 'ml', 'pyproject.toml', 'README.md', 'Init.ipynb', 'accounts.db', 'src', 'data', '.env.template', 'docs', 'model_weights', '__init__.py', 'poetry.lock', 'log']


## Configurations

In [59]:
params_grid = {
    "model" : "nlp",
    "embedder" : "fasttext",
    "embedding data" : "cc.fr.300.bin",
    "dataset" : "corpus_with_text_label_final.csv",
    "batch_size" : 300
}

In [69]:
data_folder_src = './../../data/processed/'
data_path_src = f'{data_folder_src}{params_grid["dataset"]}'
date_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
experiment_name = f'deepwoke_{params_grid["embedder"]}_{params_grid["model"]}_{date_str}'

log_dir = f"../../log/fit/{experiment_name}"
model_weight_dst = f'../../model_weights/{experiment_name}-model.keras'

# loading dataset

In [29]:

ds = pd.read_csv(data_path_src).iloc[:, 1:].dropna()
ds

,full_text,label
1,@MamzelleMNa Une très humble femme! #Ironie,0
2,"BLOGUE - «Tsé, la parité homme-femme...» au se...",0
5,Mise en ligne de mon article sur @PayeTaShnek ...,0
7,Hommage #journeedelafemme Toute sa vie elle a ...,0
8,Lustre bois flotté feuilles exotiques bleues a...,0
...,...,...
9545,"[Exclusif] #BalanceTonPorc, transparence, juge...",1
9547,"Eric Monier, directeur de la rédaction de #LCI...",1
9548,.@annesaurat porte plainte contre son ancien r...,1
9550,"Moïra Sauvage @excisionparlons ""une pensée pou...",1


## text to vector

In [30]:
def text_to_vector(text):
    words = text.split(' ')
    word_vectors = [ft.get_word_vector(word) for word in words if word in ft.words]
    if not word_vectors:
        return np.zeros(300)
    return np.mean(word_vectors, axis=0)

In [31]:
tqdm.pandas(desc='Loading data')
ds['vector'] = ds['full_text'].progress_apply(text_to_vector)

X = np.vstack(ds['vector'])
y = ds['label'].values

Loading data: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3194/3194 [06:51<00:00,  7.75it/s]


## building and training the model

### callbacks

In [54]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
    monitor='loss'
)


In [55]:

class EarlyStoppingLogging(tf.keras.callbacks.Callback):
    def __init__(self, early_stopping_callback, log_dir):
        super().__init__()
        self.early_stopping = early_stopping_callback
        self.stopped_epoch = 0
        self.writer = tf.summary.create_file_writer(log_dir)

    def on_epoch_end(self, epoch, logs=None):
        if self.early_stopping.stopped_epoch > 0:
            self.stopped_epoch = self.early_stopping.stopped_epoch
            with self.writer.as_default():
                tf.summary.scalar('early_stopping_epoch', self.stopped_epoch, step=epoch)
                self.writer.flush()


In [70]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_logging_callback = EarlyStoppingLogging(early_stopping, log_dir)

In [71]:
callback = [
    early_stopping,
    tensorboard_callback,
    hp.KerasCallback(log_dir, params_grid),
    early_stopping_logging_callback,
]

### training the model

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(128, input_dim=300, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(34, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train,
          y_train,
          epochs=500,
          batch_size = params_grid['batch_size'],
          callbacks=callback,
          validation_data=(X_test, y_test)
          )

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - accuracy: 0.8120 - loss: 0.6367 - val_accuracy: 0.8962 - val_loss: 0.4653
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8648 - loss: 0.4622 - val_accuracy: 0.8962 - val_loss: 0.3421
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8562 - loss: 0.4269 - val_accuracy: 0.8962 - val_loss: 0.3339
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8549 - loss: 0.4182 - val_accuracy: 0.8962 - val_loss: 0.3399
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8608 - loss: 0.3909 - val_accuracy: 0.8962 - val_loss: 0.3264
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8509 - loss: 0.3971 - val_accuracy: 0.8962 - val_loss: 0.3200
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8536 - loss: 0.3814 - val_accuracy: 0.8962 - val_loss: 0.3193
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8592 - loss: 0.3609 - val_accuracy: 0.8962 - val_loss:

# save model

In [ ]:
model.save(model_weight_dst)

### testing with custom sentence

In [ ]:
model_weight_dst = f'../../model_weights/fasttext_nlp_20240613-131608-model.keras'
new_model = load_model(model_weight_dst)

In [ ]:
text_nice = [
    "la bonjour je suis un type sympas", 
    "hello ça va les gens",
    "j'aime pas trop ce mec",
    "cette meuf est cool",
    "les meuf sont trop cool",
    "les mecs sont des gens stylé",
    "le mec est stylé",
    "nan mais mon manager exagère quand il dit que je suis pas assez productif et que je travail pas assez, franchement il abuse",
    "les hommes cis sont toxiques pour la société car ils imposent leur vision paternaliste",
    "Les hommes sont moins matures que les femmes quand il s'agit de prendre des décisions"
    
]

text_no_nice = [
    "regarde moi cette connasse",
    "les femme leur place c'est à la cuisine",
    "femme au volant, accident",
    "les femmes on peux pas leur faire confiance",
    "Pour une transphobe son passing est pas ouf",
    "Elle l’ouvre quand même beaucoup celle là pour quelqu’un qu’est pas capable d’éduquer ses enfants",
    "Dis donc, c’est pas parce que t’es mal baisée que tu peux tout te permettre ????",
    "Moi ce que j’aime bien chez les filles comme toi c’est que t’es pas du genre prise de tête",
    "les femmes c'est comme le fromage, quand elles puent c'est qu'elles sont bonnes",
    "Oh les grosses tepus",
    "Faut pas t'étonner si tu te fais violer vu comment t'es habillée",
    "Tu deverais avoir des enfants maintenant avant que ce soit trop tard",
    "Vous seriez une collaboratrice idéal pour notre entreprise, malheureusement nous cherchons quelqu’un disponible sur le long terme, et nous craignons qu’à votre âge des changements familiaux ne vous permettent plus de vous investir autant que besoins"
]
wordsvec_nice = [np.expand_dims(text_to_vector(text), axis=0) for text in text_nice]
wordsvec_no_nice = [np.expand_dims(text_to_vector(text), axis=0) for text in text_no_nice]

print("nice output")
for i in range(0, len(wordsvec_nice)):
    prediction = new_model.predict(wordsvec_nice[i], verbose=False)
    print(f"{text_nice[i]} : {prediction > 0.0001}")
    print(f"{prediction}")


print("\n\nnot nice output")
for i in range(0, len(wordsvec_no_nice)):
    prediction =  new_model.predict(wordsvec_no_nice[i], verbose=False)
    print(f"{text_no_nice[i]} : {prediction > 0.0001}")
    print(f"{prediction}")